In [1]:
# 导入必要的库
import os
import shutil
import random
from pathlib import Path
from typing import List, Tuple
from tqdm import tqdm
def find_annotated_images(image_dir: str, annotation_dir: str, image_extensions: List[str] = ['.jpg','.tiff', '.jpeg', '.png', '.bmp']) -> Tuple[List[Path], List[Path]]:
    """
    查找有标注和无标注的图片。

    Args:
        image_dir (str): 图片目录路径。
        annotation_dir (str): 标注文件目录路径。
        image_extensions (List[str], optional): 支持的图片扩展名列表。默认为 ['.jpg', '.jpeg', '.png', '.bmp']。

    Returns:
        Tuple[List[Path], List[Path]]: 有标注的图片路径列表和无标注的图片路径列表。
    """
    image_dir = Path(image_dir)
    annotation_dir = Path(annotation_dir)

    # 收集所有图片文件
    all_images = [p for p in image_dir.iterdir() if p.suffix.lower() in image_extensions and p.is_file()]

    annotated_images = []
    non_annotated_images = []

    for img_path in tqdm(all_images, desc="查找有标注和无标注的图片"):
        # 假设标注文件与图片同名但扩展名为 .txt
        label_path = annotation_dir / f"{img_path.stem}.json"
        if label_path.exists():
            annotated_images.append(img_path)
        else:
            non_annotated_images.append(img_path)

    return annotated_images, non_annotated_images

def split_data(annotated_images: List[Path], train_ratio: float = 0.8) -> Tuple[List[Path], List[Path]]:
    """
    将有标注的图片随机划分为训练集和验证集。

    Args:
        annotated_images (List[Path]): 有标注的图片路径列表。
        train_ratio (float, optional): 训练集比例。默认为 0.8。

    Returns:
        Tuple[List[Path], List[Path]]: 训练集图片路径列表和验证集图片路径列表。
    """
    random.shuffle(annotated_images)
    split_index = int(len(annotated_images) * train_ratio)
    train_images = annotated_images[:split_index]
    val_images = annotated_images[split_index:]
    return train_images, val_images

def organize_data(
    image_dir: str,
    annotation_dir: str,
    output_dir: str,
    annotated_images: List[Path],
    non_annotated_images: List[Path],
    train_ratio: float = 0.8
) -> None:
    """
    将图片和标注文件组织到不同的目录中，并划分训练集和验证集。

    Args:
        image_dir (str): 图片目录路径。
        annotation_dir (str): 标注文件目录路径。
        output_dir (str): 输出的组织后数据的目录路径。
        annotated_images (List[Path]): 有标注的图片路径列表。
        non_annotated_images (List[Path]): 无标注的图片路径列表。
        train_ratio (float, optional): 训练集比例。默认为 0.8。
    """
    image_dir = Path(image_dir)
    annotation_dir = Path(annotation_dir)
    output_dir = Path(output_dir)

    train_images_dir = output_dir / 'train' / 'images'
    train_labels_dir = output_dir / 'train' / 'rawlabels'
    val_images_dir = output_dir / 'val' / 'images'
    val_labels_dir = output_dir / 'val' / 'rawlabels'
    non_annotated_images_dir = output_dir / 'non_annotated_images'

    # 创建目标目录
    train_images_dir.mkdir(parents=True, exist_ok=True)
    train_labels_dir.mkdir(parents=True, exist_ok=True)
    val_images_dir.mkdir(parents=True, exist_ok=True)
    val_labels_dir.mkdir(parents=True, exist_ok=True)
    non_annotated_images_dir.mkdir(parents=True, exist_ok=True)

    # 划分训练集和验证集
    train_images, val_images = split_data(annotated_images, train_ratio)

    # 复制训练集
    for img_path in tqdm(train_images, desc="复制训练集图片"):
        shutil.copy(img_path, train_images_dir / img_path.name)
        label_path = annotation_dir / f"{img_path.stem}.json"
        if label_path.exists():
            shutil.copy(label_path, train_labels_dir / label_path.name)

    # 复制验证集
    for img_path in tqdm(val_images, desc="复制验证集图片"):
        shutil.copy(img_path, val_images_dir / img_path.name)
        label_path = annotation_dir / f"{img_path.stem}.json"
        if label_path.exists():
            shutil.copy(label_path, val_labels_dir / label_path.name)

    # 复制无标注的图片
    for img_path in tqdm(non_annotated_images, desc="复制无标注的图片"):
        shutil.copy(img_path, non_annotated_images_dir / img_path.name)

    print(f"训练集图片已复制到: {train_images_dir}")
    print(f"训练集标注文件已复制到: {train_labels_dir}")
    print(f"验证集图片已复制到: {val_images_dir}")
    print(f"验证集标注文件已复制到: {val_labels_dir}")
    print(f"无标注的图片已复制到: {non_annotated_images_dir}")


# 设置目录路径
image_dir = "/data/yan/track/TS-20241123222043114.avi_frames"          # 原始图片目录
annotation_dir = "/data/yan/track/TS-20241123222043114.avi_frames/annotations"  # 标注文件目录
output_dir = "./datasets"                                 # 输出数据集目录
train_ratio = 0.8                                        # 训练集比例

# 查找有标注和无标注的图片
annotated_images, non_annotated_images = find_annotated_images(image_dir, annotation_dir)

print(f"找到 {len(annotated_images)} 张有标注的图片。")
print(f"找到 {len(non_annotated_images)} 张无标注的图片。")

# 组织数据并划分训练集和验证集
organize_data(
    image_dir=image_dir,
    annotation_dir=annotation_dir,
    output_dir=output_dir,
    annotated_images=annotated_images,
    non_annotated_images=non_annotated_images,
    train_ratio=train_ratio
)


查找有标注和无标注的图片: 100%|██████████| 2184/2184 [00:00<00:00, 133636.19it/s]


找到 120 张有标注的图片。
找到 2064 张无标注的图片。


复制训练集图片:   0%|          | 0/96 [00:00<?, ?it/s]

复制无标注的图片: 100%|██████████| 2064/2064 [00:01<00:00, 1057.21it/s]

训练集图片已复制到: datasets/train/images
训练集标注文件已复制到: datasets/train/rawlabels
验证集图片已复制到: datasets/val/images
验证集标注文件已复制到: datasets/val/rawlabels
无标注的图片已复制到: datasets/non_annotated_images


In [2]:
import json
from pathlib import Path
from tqdm import tqdm

def convert_labelme_to_yolo_segmentation(json_dir: str, output_dir: str, classes: list):
    """
    将 Labelme 的 JSON 格式转化为 YOLO 分割格式。

    Args:
        json_dir (str): 存放 JSON 文件的目录。
        output_dir (str): 存放 YOLO 格式标注文件的目录。
        classes (list): 类别名称列表。
    """
    json_dir = Path(json_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    json_files = list(json_dir.glob("*.json"))
    
    for json_file in tqdm(json_files, desc="转换分割标注文件"):
        with open(json_file, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        img_width = data['imageWidth']
        img_height = data['imageHeight']
        yolo_annotations = []
        
        for shape in data['shapes']:
            label = shape['label']
            points = shape['points']
            
            # 归一化多边形点
            normalized_points = [
                f"{point[0] / img_width:.6f} {point[1] / img_height:.6f}" 
                for point in points
            ]
            
            if label in classes:
                class_id = classes.index(label)
                # YOLO 分割格式：<class_id> <x1> <y1> <x2> <y2> ... <xn> <yn>
                annotation = f"{class_id} " + " ".join(normalized_points)
                yolo_annotations.append(annotation)
            else:
                print(f"警告: 标签 {label} 不在 classes 列表中，已跳过。")
        
        # 输出 YOLO 格式的标注文件
        output_path = output_dir / f"{json_file.stem}.txt"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write("\n".join(yolo_annotations))
    
    print(f"所有分割标注文件已转换并保存到 {output_dir}")

# 设置目录路径和类别列表
json_dir = "./datasets/val/rawlabels"  # 标注文件 JSON 目录
output_dir = "./datasets/val/labels"  # 转换后保存 YOLO 格式的目录
classes = ["Celegans"]  # 替换为你的实际类别名称
# 转换标注文件
convert_labelme_to_yolo_segmentation(json_dir, output_dir, classes)

# 设置目录路径和类别列表
json_dir = "./datasets/train/rawlabels"  # 标注文件 JSON 目录
output_dir = "./datasets/train/labels"  # 转换后保存 YOLO 格式的目录
classes = ["Celegans"]  # 替换为你的实际类别名称
# 转换标注文件
convert_labelme_to_yolo_segmentation(json_dir, output_dir, classes)
#  pip install ultralytics -i https://pypi.tuna.tsinghua.edu.cn/simple

转换分割标注文件: 100%|██████████| 35/35 [00:00<00:00, 316.87it/s]


所有分割标注文件已转换并保存到 datasets/val/labels


转换分割标注文件: 100%|██████████| 107/107 [00:00<00:00, 337.50it/s]

所有分割标注文件已转换并保存到 datasets/train/labels


In [2]:
from ultralytics import YOLO

# 加载预训练的YOLOv11分割模型
model = YOLO('yolo11x-seg')

results = model.train(
    data='./data.yaml',
    device=[0,1,2,3],
    epochs=300,
    imgsz=1280,
    batch=16,
    lr0=0.001,
    name='yolov11_WormCNN-HTT',
    project='runs/segment',
    task='segment',
    mosaic=True,
    mixup=0.2,                # 提高 mixup 强度
    fliplr=0.5,
    hsv_h=0.02,               # 色调变换范围
    hsv_s=0.8,                # 饱和度缩放范围
    hsv_v=0.4,                # 明度缩放范围
    degrees=10,               # 随机旋转范围
    translate=0.2,            # 平移范围
    scale=0.5,                # 随机缩放范围
    # shear=10,                 # 剪切范围
    perspective=0.001,        # 透视变化
    flipud=0.1,               # 上下翻转概率
    copy_paste=True           # 启用 Copy-Paste 数据增强
)


Ultralytics 8.3.36 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
                                                       CUDA:1 (NVIDIA GeForce RTX 4090, 24210MiB)
                                                       CUDA:2 (NVIDIA GeForce RTX 4090, 24210MiB)
                                                       CUDA:3 (NVIDIA GeForce RTX 4090, 24210MiB)
engine/trainer: task=segment, mode=train, model=yolo11x-seg.pt, data=./data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=1280, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=8, project=runs/segment, name=yolov11_WormCNN-HTT28, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=N

train: Scanning /data/yan/track/datasets/train/labels.cache... 107 images, 0 backgrounds, 0 corrupt: 100%|██████████| 107/107 [00:00<?, ?it/s]
val: Scanning /data/yan/track/datasets/val/labels.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]


Plotting labels to runs/segment/yolov11_WormCNN-HTT28/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000714, momentum=0.9) with parameter groups 176 weight(decay=0.0), 187 weight(decay=0.0005), 186 bias(decay=0.0)
Image sizes 1280 train, 1280 val
Using 32 dataloader workers
Logging results to runs/segment/yolov11_WormCNN-HTT28
Starting training for 300 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      1/300      19.4G      0.758      1.199      3.478     0.9054         57       1280: 100%|██████████| 7/7 [00:03<00:00,  1.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.70it/s]


                   all         35        158    0.00419      0.278    0.00268    0.00216       0.01      0.665     0.0235    0.00727

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         35        158      0.987          1      0.994      0.937      0.987          1      0.994      0.794

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    300/300      19.1G     0.3821     0.4256     0.2024     0.8229         13       1280: 100%|██████████| 7/7 [00:02<00:00,  3.00it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  6.82it/s]


                   all         35        158      0.987          1      0.994      0.939      0.987          1      0.994      0.793

300 epochs completed in 0.354 hours.
Optimizer stripped from runs/segment/yolov11_WormCNN-HTT28/weights/last.pt, 124.9MB
Optimizer stripped from runs/segment/yolov11_WormCNN-HTT28/weights/best.pt, 124.9MB

Validating runs/segment/yolov11_WormCNN-HTT28/weights/best.pt...
Ultralytics 8.3.36 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
                                                       CUDA:1 (NVIDIA GeForce RTX 4090, 24210MiB)
                                                       CUDA:2 (NVIDIA GeForce RTX 4090, 24210MiB)
                                                       CUDA:3 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLO11x-seg summary (fused): 491 layers, 62,003,283 parameters, 0 gradients, 318.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  6.42it/s]


                   all         35        158       0.99          1      0.995      0.937       0.99          1      0.995      0.807
Speed: 0.2ms preprocess, 15.8ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/segment/yolov11_WormCNN-HTT28


In [3]:
from ultralytics import YOLO
model = YOLO("/data/yan/track/runs/segment/yolov11_WormCNN-HTT28/weights/best.pt")  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category
metrics.seg.map  # map50-95(M)
metrics.seg.map50  # map50(M)
metrics.seg.map75  # map75(M)
metrics.seg.maps  # a list contains map50-95(M) of each category



Ultralytics 8.3.36 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 24210MiB)
YOLO11x-seg summary (fused): 491 layers, 62,003,283 parameters, 0 gradients, 318.5 GFLOPs


val: Scanning /data/yan/track/datasets/val/labels.cache... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.93it/s]


                   all         35        158       0.99          1      0.995      0.941       0.99          1      0.995      0.784
Speed: 3.9ms preprocess, 28.0ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/segment/val3


array([    0.78358])

In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment
from collections import defaultdict
import csv
video_path = "/data/yan/track/vedio/TS-20241123220321351.avi"
cap = cv2.VideoCapture(video_path)

if cap.isOpened():
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"Video Resolution: Width={frame_width}, Height={frame_height}")
else:
    print("Error: Cannot open video.")

# assert mask_numpy.shape == (frame_height, frame_width), "Mask dimensions mismatch with video frame!"
# 获取分割结果的尺寸
mask_height, mask_width = mask_numpy.shape

# 计算分辨率缩放比例
scale_x = frame_width / mask_width
scale_y = frame_height / mask_height

# 矫正质心坐标
cx = int((moments["m10"] / moments["m00"]) * scale_x)
cy = int((moments["m01"] / moments["m00"]) * scale_y)

# 初始化轨迹存储
# Initialize variables
trajectories = defaultdict(list)  # {worm_id: [(frame, x, y), ...]}
next_worm_id = 0
worm_positions = {}  # {worm_id: (x, y)}

# Distance threshold (pixels)
base_distance_threshold = 50

# Exponential smoothing parameter
alpha = 0.1  # Smoothing factor between 0.0 and 1.0

def smooth_coordinates(worm_id, new_x, new_y):
    """Smooth coordinates to reduce jitter."""
    if worm_id in worm_positions:
        prev_x, prev_y = worm_positions[worm_id]
        smoothed_x = alpha * prev_x + (1 - alpha) * new_x
        smoothed_y = alpha * prev_y + (1 - alpha) * new_y
        return int(smoothed_x), int(smoothed_y)
    else:
        return new_x, new_y

# Process video frames
results = model.predict(
    video_path,
    line_width=4,
    save=True,
    imgsz=1280,
    conf=0.5,
    save_frames=True,
    save_txt=True,
    stream=True,
)

for frame_idx, result in enumerate(results):
    print(f"Processing Frame {frame_idx}...")
    current_centroids = []

    # Check if there are segmentation results
    if result.masks is None or len(result.masks.data) == 0:
        print(f"No masks detected in Frame {frame_idx}.")
        worm_positions.clear()  # Clear positions if no worms are detected
        continue

    # Get centroids of each segmentation mask
    # Get centroids of each segmentation mask
    masks = result.masks.data
    for mask in masks:
        mask_numpy = mask.cpu().numpy().astype('uint8')  # 转换为 numpy 格式
        moments = cv2.moments(mask_numpy)  # 计算图像的矩
        if moments["m00"] != 0:  # 确保区域有非零面积
            cx = int(moments["m10"] / moments["m00"])  # 计算质心 x 坐标
            cy = int(moments["m01"] / moments["m00"])  # 计算质心 y 坐标
            current_centroids.append((cx, cy))


    if frame_idx == 0:
        # Initialize worm positions and trajectories
        for centroid in current_centroids:
            worm_positions[next_worm_id] = centroid
            trajectories[next_worm_id].append((frame_idx, *centroid))
            next_worm_id += 1
    else:
        # Previous worm IDs and positions
        prev_worm_ids = list(worm_positions.keys())
        prev_positions = [worm_positions[wid] for wid in prev_worm_ids]

        # Compute distance matrix
        distances = cdist(prev_positions, current_centroids)

        # Use Hungarian algorithm for optimal assignment
        row_indices, col_indices = linear_sum_assignment(distances)

        matched_worm_ids = set()
        matched_cols = set()

        for row, col in zip(row_indices, col_indices):
            distance = distances[row, col]
            if distance < base_distance_threshold:
                worm_id = prev_worm_ids[row]
                centroid = current_centroids[col]
                # smoothed_x, smoothed_y = smooth_coordinates(worm_id, *centroid)
                worm_positions[worm_id] = (smoothed_x, smoothed_y)
                trajectories[worm_id].append((frame_idx, smoothed_x, smoothed_y))
                matched_worm_ids.add(worm_id)
                matched_cols.add(col)
            else:
                # Distance too large, do not match
                pass

        # Remove unmatched previous worm IDs (optional: keep for a few frames)
        unmatched_worm_ids = set(prev_worm_ids) - matched_worm_ids
        for worm_id in unmatched_worm_ids:
            del worm_positions[worm_id]

        # Assign new IDs to unmatched current centroids
        for idx, centroid in enumerate(current_centroids):
            if idx not in matched_cols:
                worm_id = next_worm_id
                smoothed_x, smoothed_y = smooth_coordinates(worm_id, *centroid)
                worm_positions[worm_id] = (smoothed_x, smoothed_y)
                trajectories[worm_id].append((frame_idx, smoothed_x, smoothed_y))
                next_worm_id += 1



# 保存轨迹数据
import csv

with open("worm_trajectories.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["WormID", "Frame", "X", "Y"])
    for worm_id, trajectory in trajectories.items():
        for frame_idx, x, y in trajectory:
            writer.writerow([worm_id, frame_idx, x, y])


print("Trajectories saved to 'worm_trajectories.csv'.")




Video Resolution: Width=1600, Height=1200

video 1/1 (frame 1/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 0...
video 1/1 (frame 2/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 1...
video 1/1 (frame 3/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 2...
video 1/1 (frame 4/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 3...
video 1/1 (frame 5/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 4...
video 1/1 (frame 6/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 5...
video 1/1 (frame 7/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24.1ms
Processing Frame 6...
video 1/1 (frame 8/293) /data/yan/track/vedio/TS-20241123220321351.avi: 960x1280 6 Celeganss, 24

AssertionError: Mask dimensions mismatch with video frame!

In [34]:
import pandas as pd
import cv2
import imageio
import numpy as np
from PIL import Image

# 参数设置
video_path = "/data/yan/track/vedio/TS-20241123220321351.avi"  # 视频文件路径
csv_file = "worm_trajectories.csv"  # 轨迹数据文件
output_gif = "worm_trajectories.gif"  # 输出 GIF 文件名
optimized_gif = "worm_trajectories_optimized.gif"  # 优化后的 GIF 文件名

# 定义目标分辨率
target_width, target_height = 1600, 1200  # 将分辨率减半

# 定义采样率
sampling_rate = 2  # 每隔一帧处理一次

# 读取轨迹数据
data = pd.read_csv(csv_file)

# 提取帧数范围
max_frame = data["Frame"].max()

# 打开视频以提取背景帧
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Cannot open video.")
    exit()

# 获取原始视频帧大小
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"Original Video Resolution: Width={original_width}, Height={original_height}")

# 初始化 GIF 帧列表
frames_for_gif = []

# 处理每一帧
for frame_idx in range(max_frame + 1):
    if frame_idx % sampling_rate != 0:
        # 跳过当前帧
        ret = cap.grab()
        if not ret:
            print(f"End of video reached at frame {frame_idx}.")
            break
        continue

    # 读取视频帧
    ret, frame = cap.read()
    if not ret:
        print(f"End of video reached at frame {frame_idx}.")
        break

    # 调整帧大小到目标分辨率
    frame = cv2.resize(frame, (target_width, target_height))

    # 提取当前帧的轨迹点
    current_data = data[data["Frame"] == frame_idx]

    # 在帧上绘制轨迹
    for _, row in current_data.iterrows():
        worm_id = int(row["WormID"])
        x, y = int(row["X"]), int(row["Y"])
        # 根据新的分辨率调整坐标
        x_resized = int(x * target_width / original_width)
        y_resized = int(y * target_height / original_height)
        # 根据 WormID 生成颜色
        color = (worm_id * 50 % 256, worm_id * 80 % 256, worm_id * 100 % 256)
        # 绘制小圆点表示质心
        cv2.circle(frame, (x_resized, y_resized), 3, color, -1)

    # 在帧上添加帧号
    cv2.putText(frame, f"Frame: {frame_idx}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    # 转为 RGB 格式（OpenCV 默认是 BGR）
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # 添加到 GIF 帧列表
    frames_for_gif.append(frame_rgb)

# 释放视频捕捉对象
cap.release()

# 生成 GIF，调整帧率
imageio.mimsave(output_gif, frames_for_gif, fps=5)  # 将 fps 从 10 降低到 5
print(f"GIF saved as '{output_gif}'.")

# 使用 Pillow 进行颜色优化
image = Image.open(output_gif)
image = image.convert('P', palette=Image.ADAPTIVE, colors=128)  # 将颜色数量减少到 128
image.save(optimized_gif, optimize=True)
print(f"Optimized GIF saved as '{optimized_gif}'.")


Original Video Resolution: Width=1600, Height=1200
GIF saved as 'worm_trajectories.gif'.
Optimized GIF saved as 'worm_trajectories_optimized.gif'.


In [16]:
trajectories

defaultdict(list, {})

In [5]:
import cv2
import os
import glob

def create_video_from_images(image_dir, output_video_path, fps=30):
    # 获取目录中所有的 .jpg 文件，并按文件名中的数字排序
    image_files = sorted(glob.glob(os.path.join(image_dir, "*.jpg")), key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

    if not image_files:
        raise FileNotFoundError(f"没有找到任何 .jpg 文件: {image_dir}")

    # 读取第一张图片以获取图像尺寸
    first_image = cv2.imread(image_files[0])
    height, width, layers = first_image.shape

    # 定义视频编码器和输出文件
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用 MP4 编码
    video = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 将所有图片写入视频
    for image_file in image_files:
        img = cv2.imread(image_file)
        if img is None:
            print(f"无法读取图像文件: {image_file}")
            continue
        video.write(img)

    # 释放资源
    video.release()
    print(f"视频已创建并保存到: {output_video_path}")

if __name__ == "__main__":
    # 指定包含 .jpg 文件的目录
    image_directory = '/data/yan/track/runs/segment/predict4/TS-20241123220321351_frames'  # 替换为你的图像目录路径
    # 指定输出视频文件路径
    output_video_path = 'output_video.mp4'  # 替换为你希望保存视频的路径
    # 创建视频
    create_video_from_images(image_directory, output_video_path, fps=30)


视频已创建并保存到: output_video.mp4


In [7]:
import cv2
from ultralytics import YOLO
import os
import numpy as np
import traceback

def main():
    # Load the trained model
    model_path = '/data/yan/track/runs/segment/yolov11_WormCNN-HTT22/weights/best.pt'  # Replace with your best model path
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Model file does not exist: {model_path}")
    model = YOLO(model_path)

    # Input video path
    video_path = '/data/yan/track/vedio/TS-20241123220321351.avi'  # Replace with your video path
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video file does not exist: {video_path}")
    
    # Open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video file: {video_path}")

    # Get video parameters
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Video FPS: {fps}, Resolution: {width}x{height}, Total frames: {total_frames}")

    # Define the output image save path
    output_dir = 'output_images'
    os.makedirs(output_dir, exist_ok=True)

    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        print(f"Processing frame {frame_count}/{total_frames}", end='\r')

        if frame is not None:
            # Ensure the input size matches the model's requirement
            frame_resized = cv2.resize(frame, (640, 640))
            # Print frame shape and data type
            print(f"Frame shape: {frame_resized.shape}, dtype: {frame_resized.dtype}")
        else:
            print(f"Skipping empty frame {frame_count}")
            continue

        # Perform detection
        try:
            # Adjust confidence threshold if necessary
            results_generator = model.predict(
                source=frame_resized,
                imgsz=640,
                stream=True,
                conf=0.1  # Adjust confidence threshold
            )
            
            # Since results_generator is a generator, we need to iterate over it
            for result in results_generator:
                boxes = result.boxes
                if boxes:
                    print(f"Frame {frame_count}: {len(boxes)} detections")
                    for box in boxes:
                        # Print detection details
                        print(f"Detection: Class {box.cls}, Confidence {box.conf}")
                else:
                    print(f"Frame {frame_count}: No detections")
                
                # Draw detection results
                annotated_frame = result.plot()

                # Save the image
                output_image_path = os.path.join(output_dir, f"frame_{frame_count:06d}.jpg")
                cv2.imwrite(output_image_path, annotated_frame)
        except Exception as e:
            print(f"Error processing frame {frame_count}: {e}")
            traceback.print_exc()
            continue

    # Release resources
    cap.release()
    print(f"\nProcessing complete, output images saved in: {output_dir}")

if __name__ == "__main__":
    main()


Video FPS: 29.3, Resolution: 1600x1200, Total frames: 293
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 1: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 2: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 3: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 4: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 5: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 6: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 7: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 8: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 9: too many values to

Traceback (most recent call last):
  File "/tmp/ipykernel_2239786/1301476628.py", line 65, in main
    for result in results_generator:
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 36, in generator_context
    response = gen.send(None)
               ^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/engine/predictor.py", line 266, in stream_inference
    self.results = self.postprocess(preds, im, im0s)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/models/yolo/segment/predict.py", line 52, in postprocess
    masks = ops.process_mask(proto[i], pred[:, 6:], pred[:, :4], img.shape[2:], upsample=True)  # HWC
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/utils/ops.py", line 6

Error processing frame 13: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 14: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 15: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 16: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 17: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 18: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 19: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 20: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 21: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 22: t

Traceback (most recent call last):
  File "/tmp/ipykernel_2239786/1301476628.py", line 65, in main
    for result in results_generator:
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 36, in generator_context
    response = gen.send(None)
               ^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/engine/predictor.py", line 266, in stream_inference
    self.results = self.postprocess(preds, im, im0s)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/models/yolo/segment/predict.py", line 52, in postprocess
    masks = ops.process_mask(proto[i], pred[:, 6:], pred[:, :4], img.shape[2:], upsample=True)  # HWC
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/utils/ops.py", line 6

Error processing frame 25: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 26: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 27: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 28: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 29: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 30: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 31: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 32: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 33: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 34: t

...

Traceback (most recent call last):
  File "/tmp/ipykernel_2239786/1301476628.py", line 65, in main
    for result in results_generator:
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 36, in generator_context
    response = gen.send(None)
               ^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/engine/predictor.py", line 266, in stream_inference
    self.results = self.postprocess(preds, im, im0s)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/models/yolo/segment/predict.py", line 52, in postprocess
    masks = ops.process_mask(proto[i], pred[:, 6:], pred[:, :4], img.shape[2:], upsample=True)  # HWC
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/utils/ops.py", line 6

Error processing frame 229: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 230: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 231: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 232: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 233: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 234: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 235: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 236: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 237: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing fr

Traceback (most recent call last):
  File "/tmp/ipykernel_2239786/1301476628.py", line 65, in main
    for result in results_generator:
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 36, in generator_context
    response = gen.send(None)
               ^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/engine/predictor.py", line 266, in stream_inference
    self.results = self.postprocess(preds, im, im0s)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/models/yolo/segment/predict.py", line 52, in postprocess
    masks = ops.process_mask(proto[i], pred[:, 6:], pred[:, :4], img.shape[2:], upsample=True)  # HWC
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/utils/ops.py", line 6

Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 241: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 242: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 243: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 244: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 245: too many values to unpack (expected 3)
Frame shape: (640, 640, 3), dtype: uint8

Error processing frame 246: too many values to unpack (expected 3)

Processing complete, output images saved in: output_images


Traceback (most recent call last):
  File "/tmp/ipykernel_2239786/1301476628.py", line 65, in main
    for result in results_generator:
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/torch/utils/_contextlib.py", line 36, in generator_context
    response = gen.send(None)
               ^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/engine/predictor.py", line 266, in stream_inference
    self.results = self.postprocess(preds, im, im0s)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/models/yolo/segment/predict.py", line 52, in postprocess
    masks = ops.process_mask(proto[i], pred[:, 6:], pred[:, :4], img.shape[2:], upsample=True)  # HWC
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/spuser/.conda/envs/yanpan/lib/python3.11/site-packages/ultralytics/utils/ops.py", line 6